In [40]:
import pandas as pd
import numpy as np

In [41]:
dataset = pd.read_csv('Data/IBTrACS/ibtrac.csv')

In [42]:
#------------------------------------------------------------------------------------------#
LAT_MIN         = 13.0 # degrees North
LAT_MAX         = 30.0
LON_MIN         = 120.0 # degrees East
LON_MAX         = 153.0
#------------------------------------------------------------------------------------------#

In [43]:
dataset['A_VMAX'] = 0.1884 * dataset['VMAX']**1.288
dataset['VMAX2'] = 0
dataset.loc[dataset['YEAR'] < 1968, 'VMAX2'] = 0.4 * dataset['VMAX'] + 0.6 * dataset['A_VMAX']
dataset.loc[dataset['YEAR'].between(1968,1972), 'VMAX2'] = 0.8 * dataset['VMAX'] + 0.2 * dataset['A_VMAX']
dataset.loc[dataset['YEAR'] > 1972, 'VMAX2'] = dataset['VMAX']

# Determine max wind of the each cyclone
max_wind = dataset.groupby('CycNo')['VMAX2'].max().rename('VMAX2').reset_index()

# Remove cyclones with maximum windspeed lower than typhoon 64 knots
max_wind = max_wind[max_wind['VMAX2']>64]

# Keep relevant cyclones
dataset = dataset[dataset['CycNo'].isin(max_wind['CycNo'])]

In [37]:
def region_check(row):
    lat = row['LAT']
    lon = row['LON']
    
    if lat <= LAT_MAX and lat >= LAT_MIN and lon <= LON_MAX and lon >= LON_MIN:
        return True
    else:
        return False

dataset['REGION'] = dataset.apply(region_check, axis = 1)

def region_check2(series):
    if any(series['REGION']):
        return True
    else:
        return False

region_result = dataset.groupby('CycNo').apply(region_check2)

dataset = dataset.merge(region_result.rename('REGION2').reset_index(), on ='CycNo', how='inner')
# Select only data in the region
dataset = dataset[dataset['REGION2']==True]

In [44]:
dataset = dataset.drop('VMAX', axis=1)
dataset = dataset.rename(columns={'VMAX2': 'VMAX'})

In [45]:
dataset['CycNo'].nunique()

939

In [32]:
dataset.to_csv('clean_ibtrac.csv', index=False)